In [16]:
import numpy as np
import pandas as pd
import os
import json
import gzip
import pandas as pd
from urllib.request import urlopen
from tqdm import tqdm 
def parse(path):
  g = gzip.open(path, 'r')
  for l in g:
    yield json.loads(l)

In [17]:
import pandas as pd
import gzip
import json
import random 
def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield json.loads(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

In [18]:
import pickle as pkl

test_datas = pd.read_pickle('../data/ML_1M/test_ood2.pkl')
test_datas["uid"] = test_datas["uid"].apply(lambda x:int(x))
test_datas["iid"] = test_datas["iid"].apply(lambda x:int(x))
length = len(test_datas)
history_length = []
for i in range(length):
    history_length.append(len(test_datas.iloc[i].his)-1)
test_datas["his_length"] = history_length
print(len(test_datas))
cold_data = 0
for i in range(length):
    if test_datas.iloc[i].his_length <=4:
        cold_data += 1
print(cold_data)
test_datas.head()

7331
421


,uid,iid,label,timestamp,his,his_title,title,flag,not_cold,his_length
97,4,96,1,1040544350,"[0, 38, 39, 40, 42, 43, 44, 45, 46, 47, 48, 50...","[, Star Wars: Episode IV - A New Hope (1977), ...",Deconstructing Harry (1997),1.0,1,52
98,4,97,1,1040544350,"[0, 38, 39, 40, 42, 43, 44, 45, 46, 47, 48, 50...","[, Star Wars: Episode IV - A New Hope (1977), ...",Indecent Proposal (1993),1.0,1,53
99,4,98,0,1040544494,"[0, 38, 39, 40, 42, 43, 44, 45, 46, 47, 48, 50...","[, Star Wars: Episode IV - A New Hope (1977), ...",Big Daddy (1999),1.0,1,54
100,4,99,1,1040544521,"[0, 38, 39, 40, 42, 43, 44, 45, 46, 47, 48, 50...","[, Star Wars: Episode IV - A New Hope (1977), ...",But I'm a Cheerleader (1999),1.0,1,54
101,4,100,1,1040544564,"[0, 38, 39, 40, 42, 43, 44, 45, 46, 47, 48, 50...","[, Star Wars: Episode IV - A New Hope (1977), ...",Desperately Seeking Susan (1985),1.0,1,55


In [19]:
import pickle as pkl

train_datas = pd.read_pickle('../data//ML_1M/train_ood2.pkl')

length = len(train_datas)
history_length = []
for i in range(length):
    history_length.append(len(train_datas.iloc[i].his)-1)
train_datas["his_length"] = history_length


In [20]:

train_datas = train_datas.sort_values(by=["uid","timestamp"])
train_datas["uid"] = train_datas["uid"].apply(lambda x:int(x))
train_datas["iid"] = train_datas["iid"].apply(lambda x:int(x))
filtered_index = []
for i in tqdm(range(len(train_datas))):
    if len(train_datas.iloc[i].his) >= 2 and len(train_datas.iloc[i].his) <= 21:
        filtered_index.append(i)
train_datas = train_datas.iloc[filtered_index]
print("after filter train datas number: ", len(train_datas))
print("after filter train datas user id number: ", len(train_datas.uid.unique()))


100%|██████████| 33891/33891 [00:03<00:00, 10655.10it/s]

after filter train datas number:  14760
after filter train datas user id number:  671


In [21]:
test_filtered_index = []
for i in tqdm(range(len(test_datas))):
    if len(test_datas.iloc[i].his) >=2 and len(test_datas.iloc[i].his) <=21:
        test_filtered_index.append(i)
test_datas = test_datas.iloc[test_filtered_index]
print("after filter test datas number: ",len(test_datas))
print("after filter test datas user id number:", len(test_datas.uid.unique()))

cold_user_id = set()
for i in range(len(test_datas)):
    if test_datas.iloc[i].not_cold == 0:
        cold_user_id.add(int(test_datas.iloc[i].uid))

cold_user_dict = {}
cold_user_dict["cold_users_id"] = list(cold_user_id)


100%|██████████| 7331/7331 [00:00<00:00, 10311.46it/s]


after filter test datas number:  1865
after filter test datas user id number: 166


In [22]:
total_datas = pd.concat([train_datas, test_datas],axis=0)
total_datas = total_datas.sort_values(by=["uid","timestamp"])
user_last_index = []
uid = 1
for i in range(len(total_datas)):
    if total_datas.iloc[i].uid == uid :
        continue
    else:
        user_last_index.append(int(uid))
        uid = total_datas.iloc[i].uid
user_last_index.append(int(uid))
print(len(user_last_index))

user_squences_id = {}
for id in user_last_index:
   
    user_squences_id[str(id)] = [id] + [int(item) for item in total_datas.iloc[i].his[1:]]
    if len(user_squences_id[str(id)]) < 2:
        user_squences_id[str(id)] += [0]
def save_json(data, file):
    with open(file, "w") as f:
        json.dump(data, f)

save_json(user_squences_id, "../data/movielen_for_train/user_sequences_id.json")
save_json(cold_user_dict, "../data/movielen_for_train/cold_users.json")
print("cold user number: ", len(cold_user_id))

729
cold user number:  54


### DIN dataset

In [23]:
train_datas.head()

,uid,iid,label,timestamp,his,his_title,title,flag,not_cold,his_length
1,1,2,1,994556636,"[0, 1]","[, Shawshank Redemption, The (1994)]","Sting, The (1973)",-1.0,1,1
3,2,4,0,1009669071,"[0, 3]","[, American Beauty (1999)]",Gone in 60 Seconds (2000),-1.0,1,1
4,2,5,0,1009669071,"[0, 3]","[, American Beauty (1999)]","Apostle, The (1997)",-1.0,1,1
5,2,6,0,1009669071,"[0, 3]","[, American Beauty (1999)]",Booty Call (1997),-1.0,1,1
6,2,7,0,1009669115,"[0, 3]","[, American Beauty (1999)]",Star Trek III: The Search for Spock (1984),-1.0,1,1


In [24]:

train_dataset = train_datas[["uid",'iid','label',"his"]]
train_dataset.head()
train_dataset.to_csv('../data/movielen/DIN_train.csv',index=None)


test_dataset = test_datas[["uid","iid","label","his"]]
test_dataset.to_csv('../data/movielen/DIN_test.csv', index=None)

In [30]:
all_item_ids = set()
all_user_ids = set()

user_items_count = {} ### the item  each user purchased
item_users_count = {} ### each item has been purchased by whom

id_2_title = {}
title_2_id = {}
for i in tqdm(range(len(train_datas))):
    all_user_ids.add(train_datas.iloc[i].uid)
    user_items_count[train_datas.iloc[i].uid] = user_items_count.get(train_datas.iloc[i].uid, set())
    his_ids = train_datas.iloc[i].his + [train_datas.iloc[i].iid]
    his_title = train_datas.iloc[i].his_title + [train_datas.iloc[i].title]
    for j in range(len(his_ids)):
        all_item_ids.add(his_ids[j])
        title_2_id[his_title[j]] = int(his_ids[j]) 
        id_2_title[his_ids[j]] = his_title[j]
        

        ### last item is candidate 
        if j != len(his_ids)-1:
            user_items_count[train_datas.iloc[i].uid].add(his_ids[j])
            item_users_count[his_ids[j]] = item_users_count.get(his_ids[j], set())
            item_users_count[his_ids[j]].add(train_datas.iloc[i].uid)
       

for i in tqdm(range(len(test_datas))):
    all_user_ids.add(test_datas.iloc[i].uid)
    his_ids = test_datas.iloc[i].his + [test_datas.iloc[i].iid]
    his_title = test_datas.iloc[i].his_title + [test_datas.iloc[i].title]
    for j in range(len(his_ids)):
        all_item_ids.add(his_ids[j])
        id_2_title[his_ids[j]] = his_title[j]
        title_2_id[his_title[j]] = int(his_ids[j]) 

        if j != len(his_ids)-1:
            user_items_count[train_datas.iloc[i].uid].add(his_ids[j])
            item_users_count[his_ids[j]] = item_users_count.get(his_ids[j], set())
            item_users_count[his_ids[j]].add(train_datas.iloc[i].uid)
       

print("total itmes: ", len(all_item_ids))
print("total users: ", len(all_user_ids))
print("length of id_2_title: ", len(id_2_title))
save_json(title_2_id, "../data/movielen_for_train/title_2_id.json")




  0%|          | 0/14760 [00:00<?, ?it/s]

100%|██████████| 1865/1865 [00:02<00:00, 685.46it/s]

total itmes:  2643
total users:  729
length of id_2_title:  2643


In [31]:
user_total_purchase = 0
item_total_purchase = 0 
for user in user_items_count:
    item_total_purchase += len(user_items_count[user])

for item in item_users_count:
    user_total_purchase += len(item_users_count[item])

print("Avg user purchase: ", item_total_purchase / len(user_items_count), " items.")
print("Avg item purchased by ", user_total_purchase / len(item_users_count), " users.")


Avg user purchase:  29.448584202682564  items.
Avg item purchased by  7.4763526295875895  users.


### MF Dataset

In [ ]:
train_dataset = train_datas[["uid",'iid','label']]
train_dataset.head()
train_dataset.to_csv('../data/movielen/MF_train.csv',index=None)


test_dataset = test_datas[["uid","iid","label"]]
test_dataset.to_csv('../data/movielen/MF_test.csv', index=None)

### CKF Dataset

In [ ]:
filter_data = train_datas[["uid",'iid',"label","his_title","title", "his"]]
print(len(filter_data.uid.unique()))
filter_data.head(20)

In [ ]:
def generate_sequential_recommend_prompt(data):

    
    prompt = f"""The user's preferences are encoded in <unk>, he has interacted with the following items:{data["history_list"]} recently.
    Based on this information, I want you to select the user's favorite item in the list of candidate items following: {data["candidate_item_title"]}. Your response should be a item's title in the list of candidate items.
"""
    return {"input":prompt, 'output': data['output'], "user_id":data['user_id'], "candidate_item":data["candidate_item"], "task_type":"sequential", "history_items_id":data["history_items_id"]}

def generate_ctr_prompt(data):

    prompt = f"""The user's preferences are encoded in <unk> , additionally, he has recently interacted the following items he likes {data["history_list"]}.
    Based on the user's preferences and items he liked, I want you to determine if the user will like the item whose title is: "{data["candidate_item_title"]} and its encoded feature is <unk>.". Your response should be a simple "Yes" or "No" without any explanations.
"""
    return {"input":prompt, 'output': data['output'], "user_id":data['user_id'], "candidate_item":data["candidate_item"], "task_type":"ctr", "history_items_id":data["history_items_id"]}

def generate_rating_prompt(data):

    prompt = f"""The user whose preferences is <unk>  rated the following items he has interacted: {data["history_list"]}. 
    Based on the user's rating, now given your item: {data["candidate_item_title"]} with encoded feature is <unk>, you need to predict how will user rate for this item.Your response should be a number between 1-5 where 1 being lowest and 5 being highest.
"""
    return {"input":prompt, "output":data["output"], "user_id":data["user_id"], "candidate_item":data["candidate_item"], "task_type":"rating", "history_items_id":data["history_items_id"]}


### ctr task train data

In [ ]:
from tqdm import tqdm
dict_datas_ctr = []
for i in tqdm(range(len(train_datas))):
    tmp = {}
    tmp["history_list"] = filter_data.iloc[i].his_title[1:]
    if len(tmp["history_list"]) == 0:continue
    tmp["user_id"] = filter_data.iloc[i].uid
    tmp["candidate_item_title"] = filter_data.iloc[i].title
    tmp["candidate_item"] = [filter_data.iloc[i].iid] + [0] * 9
    tmp["output"] = "Yes." if filter_data.iloc[i].label == 1 else "No."
    tmp["history_items_id"] = filter_data.iloc[i].his[1:]
    dict_datas_ctr.append(tmp)

ctr_datas = []
for data in tqdm(dict_datas_ctr):
    ctr_datas.append(generate_ctr_prompt(data))

print(ctr_datas[100])
print("ctr data: ",len(ctr_datas))

### ctr test data 

In [ ]:
from tqdm import tqdm
dict_datas_ctr = []
for i in tqdm(range(len(test_datas))):
    tmp = {}
    tmp["history_list"] = test_datas.iloc[i].his_title[1:]
    if len(tmp["history_list"]) == 0:continue
    tmp["user_id"] = test_datas.iloc[i].uid
    tmp["candidate_item_title"] = test_datas.iloc[i].title
    tmp["candidate_item"] = [test_datas.iloc[i].iid] + [0] * 9
    tmp["output"] = "Yes." if test_datas.iloc[i].label == 1 else "No."
    tmp["history_items_id"] = test_datas.iloc[i].his[1:]
    dict_datas_ctr.append(tmp)

ctr_datas_test = []
for data in tqdm(dict_datas_ctr):
    ctr_datas_test.append(generate_ctr_prompt(data))

print(ctr_datas_test[100])
print("ctr test data: ", len(ctr_datas_test))



### Topk data

In [ ]:
positive_datas = train_datas[(train_datas.label==1)]
positive_datas


In [ ]:
dict_datas_seq = []

for i in tqdm(range(len(positive_datas))):
    for j in range(1,3):
        tmp = {}
        tmp["user_id"] = positive_datas.iloc[i].uid
        tmp["history_list"] = positive_datas.iloc[i].his_title[1:-j]
        tmp["history_items_id"] = positive_datas.iloc[i].his[1:-j]
        sample_negative_ids =  random.sample(all_item_ids, 9)
        candidate_item_ids = [positive_datas.iloc[i].iid] + sample_negative_ids
        random.shuffle(candidate_item_ids)
        tmp["candidate_item"] = candidate_item_ids
        tmp["candidate_item_title"] = ["item title: " + id_2_title[item] + ", and its encoded feature is <unk>" for item in candidate_item_ids]
        tmp["output"] = positive_datas.iloc[i].title
        dict_datas_seq.append(tmp)
    
seq_datas = []
for data in tqdm(dict_datas_seq):
    seq_datas.append(generate_sequential_recommend_prompt(data))

print(seq_datas[100])
print("topk data: ", len(seq_datas))

### TopK test dataset

In [ ]:
positive_datas = test_datas[(test_datas.label==1)]
print(len(positive_datas))
print(len(test_datas))
dict_datas_seq = []
for i in tqdm(range(len(positive_datas))):
    tmp = {}
    tmp["user_id"] = positive_datas.iloc[i].uid
    tmp["history_list"] = positive_datas.iloc[i].his_title[1:]
    tmp["history_items_id"] = positive_datas.iloc[i].his[1:]
    sample_negative_ids =  random.sample(all_item_ids, 9)
    candidate_item_ids = [positive_datas.iloc[i].iid] + sample_negative_ids
    random.shuffle(candidate_item_ids)
    tmp["candidate_item"] = candidate_item_ids
    tmp["candidate_item_title"] = ["item title: " + id_2_title[item] + ", and its encoded feature is <unk>" for item in candidate_item_ids]
    tmp["output"] = positive_datas.iloc[i].title
    dict_datas_seq.append(tmp)
    
seq_datas_test = []
for data in tqdm(dict_datas_seq):
    seq_datas_test.append(generate_sequential_recommend_prompt(data))

print(seq_datas_test[100])
print("seq test data: ", len(seq_datas_test))

### Rating data

In [ ]:
rating = pd.read_csv("../data/ML_1M/ratings.dat",header=None, sep='::', names=['uid','iid','rating','timestamp'],engine='python')
item_info = pd.read_csv("../data/ML_1M/movies.dat",sep='::',encoding='latin-1',engine='python',names=['iid','title','genres'])
rating_title = pd.merge(rating, item_info, on='iid', how='inner')
rating_title_new = rating_title.sort_values(by=["uid", "timestamp"])
rating_title_new["uid"] =  rating_title_new["uid"].apply(lambda x:int(x))
rating_title_new["iid"] =  rating_title_new["iid"].apply(lambda x:int(x))

In [ ]:
data_min = pd.to_datetime(rating_title_new.timestamp, unit='s').min()
data_max = pd.to_datetime(rating_title_new.timestamp, unit='s').max()
data_min, data_max

In [ ]:
rating_title_new["time"] = pd.to_datetime(rating_title_new.timestamp, unit='s').map(lambda x: (x.year - data_min.year)*12 + x.month)

In [ ]:
rating_title_new["time"] =  rating_title_new["time"] - rating_title_new["time"].min()

In [ ]:
train_slot = list(range(14,24))
valid_slot = list(range(24,29))
test_slot = list(range(29,34))


rating_title_new["label"] = rating_title_new["rating"].apply(lambda x:1 if x>=4 else 0)

rating_train = rating_title_new[rating_title_new['time'].isin(train_slot)].copy()
rating_valid = rating_title_new[rating_title_new['time'].isin(valid_slot)].copy()
rating_test = rating_title_new[rating_title_new['time'].isin(test_slot)].copy()


rating_train.shape, rating_valid.shape, rating_test.shape

In [ ]:
users = rating_train.uid.unique()
items = rating_train.iid.unique()
users_map = dict(zip(users, np.arange(users.shape[0])+1))
items_map = dict(zip(items, np.arange(items.shape[0])+1))
users_map[0]=0
items_map[0]=0

rating_train['uid'] = rating_train['uid'].map(users_map)
rating_train['iid'] = rating_train['iid'].map(items_map)

user_ratings = {}
for i in tqdm(range(len(rating_train))):
    user_ratings[rating_train.iloc[i].uid] = user_ratings.get(rating_train.iloc[i].uid, [])
    tmp = {}
    tmp[rating_train.iloc[i].title] = rating_train.iloc[i].rating
    user_ratings[rating_train.iloc[i].uid].append(tmp)
len(user_ratings)

In [ ]:
rating_train.head()

In [ ]:
user_ratings[740]

In [ ]:
from copy import deepcopy


dict_data_rating = []


# prompt = f"""The user whose preferences is <unk>  rated the following items he has interacted: {data["history_list"]}. 
#     Based on the user's rating, now given your item: {data["candidate_item_title"]} with encoded feature is <unk>, you need to predict how will user rate for this item.Your response should be a number between 1-5 where 1 being lowest and 5 being highest.
# """
   

for user_id in range(1,len(user_ratings)+1):
    user_rating_list = user_ratings[user_id]
    if 2<= len(user_rating_list) <= 10:
        tmp = {}
        history_list , target_rating = user_rating_list[:-1], user_rating_list[-1]
        history_ = []
        history_title = []
        desc = ''
        for item in history_list:
            desc += ' '
            desc += list(item.items())[0][0]
            desc += ', rating:'
            desc += str(list(item.items())[0][1])
        history_.append(deepcopy(desc.strip(',')))
        tmp['history_list'] = history_
        tmp["user_id"] = user_id
        candidate_item_title = list(target_rating.items())[0][0]
        try:
            candidate_item_id = [title_2_id[candidate_item_title]] + [0]*9
        except:
            continue
        output = str(list(target_rating.items())[0][1])
        tmp["candidate_item_title"] = candidate_item_title
        tmp["candidate_item"] = candidate_item_id
        tmp["output"] = output
        tmp["history_items_id"] = [title_2_id.get(list(item.keys())[0], 0) for item in history_list]
        dict_data_rating.append(tmp)
    
    else:
        for i in range(min(len(user_rating_list)-11,45)):
            tmp = {}
            history_list , target_rating = user_rating_list[i:i+10], user_rating_list[i+10]
            history_ = []
            desc = ""
            for item in history_list:
                desc += "\t"
                desc += list(item.items())[0][0]
                desc += ', rating: '
                desc += str(list(item.items())[0][1])
            history_.append(desc.strip(','))
            tmp['history_list'] = history_
            tmp["user_id"] = user_id
            candidate_item_title = list(target_rating.items())[0][0]
            try:
                candidate_item_id = [title_2_id[candidate_item_title]] + [0]*9
            except:
                continue
            output = str(list(target_rating.items())[0][1])
            tmp["candidate_item_title"] = candidate_item_title
            tmp["candidate_item"] = candidate_item_id
            tmp["output"] = output
            tmp["history_items_id"] = [title_2_id.get(list(item.keys())[0], 0) for item in history_list]
            dict_data_rating.append(tmp)
print(len(dict_data_rating))
    

In [ ]:
rating_datas = []

for data in tqdm(dict_data_rating):
    rating_datas.append(generate_rating_prompt(data))
random.shuffle(rating_datas)

In [ ]:
rating_datas[100]

In [ ]:
final_train_data = ctr_datas + random.sample(seq_datas, len(ctr_datas)) + rating_datas[: len(ctr_datas)//2]
test_datas = ctr_datas_test + seq_datas_test + rating_datas[len(ctr_datas)//2:]
print("total train datas: ", len(final_train_data))
for data in final_train_data:
    data["user_id"] = int(data["user_id"])
    for key in data:

        if key in [ "candidate_item", "history_items_id"]:
            data[key] = [int(item) for item in data[key]]
            

In [ ]:
for data in test_datas:
    data["user_id"] = int(data["user_id"])
    for key in data:
        if key in [ "candidate_item", "history_items_id"]:
            data[key] = [int(item) for item in data[key]]

In [ ]:
import jsonlines
with jsonlines.open("../data/movielen_for_train/train.jsonl", 'w') as w:
    w.write_all(final_train_data)

with jsonlines.open('../data/movielen_for_train/test.jsonl', 'w') as w:
    w.write_all(test_datas)
print("total test datas: ", len(test_datas))

## All title

In [ ]:
def generate_sequential_recommend_prompt(data):

    
    prompt = f"""The user has interacted with the following items:{data["history_list"]} recently.
    Based on this information, I want you to select the user's favorite item in the list of candidate items following: {data["candidate_item_title"]}. Your response should be a item's title in the list of candidate items.
"""
    return {"input":prompt, 'output': data['output'], "user_id":data['user_id'], "candidate_item":data["candidate_item"], "task_type":"sequential", "history_items_id":data["history_items_id"]}

def generate_ctr_prompt(data):

    prompt = f"""The user has recently interacted the following items he likes {data["history_list"]}.
    Based on the user's preferences and items he liked, I want you to determine if the user will like the item whose title is: "{data["candidate_item_title"]}". Your response should be a simple "Yes" or "No" without any explanations.
"""
    return {"input":prompt, 'output': data['output'], "user_id":data['user_id'], "candidate_item":data["candidate_item"], "task_type":"ctr", "history_items_id":data["history_items_id"]}

def generate_rating_prompt(data):

    prompt = f"""The user rated the following items he has interacted: {data["history_list"]}. 
    Based on the user's rating, now given your item: {data["candidate_item_title"]} you need to predict how will user rate for this item.Your response should be a number between 1-5 where 1 being lowest and 5 being highest.
"""
    return {"input":prompt, "output":data["output"], "user_id":data["user_id"], "candidate_item":data["candidate_item"], "task_type":"rating", "history_items_id":data["history_items_id"]}

In [ ]:

from tqdm import tqdm
dict_datas_ctr = []
for i in tqdm(range(len(train_datas))):
    tmp = {}
    tmp["history_list"] = filter_data.iloc[i].his_title[1:]
    if len(tmp["history_list"]) == 0:continue
    tmp["user_id"] = filter_data.iloc[i].uid
    tmp["candidate_item_title"] = filter_data.iloc[i].title
    tmp["candidate_item"] = [filter_data.iloc[i].iid] + [0] * 9
    tmp["output"] = "Yes." if filter_data.iloc[i].label == 1 else "No."
    tmp["history_items_id"] = filter_data.iloc[i].his[1:]
    dict_datas_ctr.append(tmp)

ctr_datas = []
for data in tqdm(dict_datas_ctr):
    ctr_datas.append(generate_ctr_prompt(data))
print(len(ctr_datas))
print(ctr_datas[100])

In [ ]:
positive_datas = train_datas[(train_datas.label==1)]
positive_datas

In [ ]:
dict_datas_seq = []

for i in tqdm(range(len(positive_datas))):
    for j in range(1,3):
        tmp = {}
        tmp["user_id"] = positive_datas.iloc[i].uid
        tmp["history_list"] = positive_datas.iloc[i].his_title[1:-j]
        tmp["history_items_id"] = positive_datas.iloc[i].his[1:-j]
        sample_negative_ids =  random.sample(all_item_ids, 9)
        candidate_item_ids = [positive_datas.iloc[i].iid] + sample_negative_ids
        random.shuffle(candidate_item_ids)
        tmp["candidate_item"] = candidate_item_ids
        tmp["candidate_item_title"] = [ id_2_title[item]  for item in candidate_item_ids]
        tmp["output"] = positive_datas.iloc[i].title
        dict_datas_seq.append(tmp)
    
seq_datas = []
for data in tqdm(dict_datas_seq):
    seq_datas.append(generate_sequential_recommend_prompt(data))

print(seq_datas[100])
print("topk data: ", len(seq_datas))

In [ ]:
all_title_datas = ctr_datas + random.sample(seq_datas, len(ctr_datas)) + random.sample(rating_datas, len(ctr_datas)//2)
for data in all_title_datas:
    data["user_id"] = int(data["user_id"])
    for key in data:
        if key in [ "candidate_item", "history_items_id"]:
            data[key] = [int(item) for item in data[key]]

with jsonlines.open("../data/movielen_for_train/all_title_train.jsonl", 'w') as w:
    w.write_all(all_title_datas)
